In [8]:
import numpy as np
import util
import magnetic_field
import injector
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
class MagneticMirror2D(object):

    #######################################################################
    # Begin physical parameters                                           #
    #######################################################################

    # domain size in meter
    Lx = 3e-5 # r direction
    Lz = 1e-4

    # spatial resolution in number of cells
    Nx = int(2**4)
    Nz = int(2**5)

    # mirror ratio
    R = 5.0
    B_max = 30.0  # T

    # desired electron beta in mirror throat
    beta = 0.03

    # desired electron plasma to cyclotron frequency ratio
    w_pe_to_w_ce = 1.22

    # use a reduced ion mass for faster simulations
    m_ion = 400.0 * util.constants.m_e
    # m_ion = util.constants.m_p

    # we set the voltage on the divertor side (in units of Te) since that is
    # ultimately what we want to look at
    V_divertor = -2.5

    # initial seed macroparticle density
    nppc_seed = 5  # 800

    # total simulation time in ion thermal crossing times
    crossing_times = 1

    #######################################################################
    # End global user parameters and user input                           #
    #######################################################################

    def __init__(self):

        # the initial electron density is calculated to give the desired ratio
        # of plasma to cyclotron frequency
        self.n0 = (
            (self.w_pe_to_w_ce * self.B_max / util.constants.c)**2
            / ( util.constants.mu_0 * util.constants.m_e)
        )

        # calculate the electron temperature to get the desired beta in the
        # mirror throat
        # p=nT (T in unit of joule), and p_mag = B^2/2\mu_0
        self.Te = util.J_to_eV(
            self.beta * self.B_max**2 / (2.0 * util.constants.mu_0 * self.n0)
        )
        # electron and ion temperatures are taken to be equal
        self.Ti = self.Te

        # calculate the electron inertial length
        self.d_e = util.inertial_length(self.n0)

        # calculate the domain size in m
#         self.Lx = self.Lx * self.d_e
#         self.Lz = self.Lz * self.d_e

        self.dx = self.Lx / self.Nx
        self.dz = self.Lz / self.Nz

        # the given mirror ratio can only be achieved with the given Lz and
        # B_max for a unique coil radius
        self.R_coil = 0.5 * self.Lz / np.sqrt(self.R**(2.0/3.0) - 1.0)
        self.coil = magnetic_field.CoilBField(R=self.R_coil, B_max=self.B_max)
        # self.coil.plot_field(self.Lx/2.0/self.R_coil, self.Lz/2.0/self.R_coil)

        # calculate electron plasma frequency
        w_pe = util.plasma_freq(self.n0)

        # self.dt = 0.07 / w_pe
        # simulation timestep from electron CFL
        self.dt = self.dz / (5.0 * util.thermal_velocity(self.Te, util.constants.m_e))

        # calculate the ion crossing time to get the total simulation time
        ion_crossing_time = self.Lz / \
            util.thermal_velocity(self.Ti, self.m_ion)
        self.total_steps = int(np.ceil(
            self.crossing_times * ion_crossing_time / self.dt
        ))
        self.diag_steps = int(self.total_steps / 20.0)

        # for debug use
        self.total_steps = 100
        self.diag_steps = 10

        # calculate the flux from the thermal plasma reservoir
        # self.flux_e = (
        #     self.n0 * util.thermal_velocity(self.Te) / np.sqrt(2.0 * np.pi)
        # )
        # self.flux_i = self.flux_e * np.sqrt(util.constants.m_e / self.m_ion)
        self.flux_i = (
            self.n0 * util.thermal_velocity(self.Ti, self.m_ion)
        )

        # sanity checks
        # check spatial resolution
        self.debye_length = util.Debye_length(self.Te, self.n0)
#         assert self.dz < self.debye_length

#         assert np.isclose(self.coil.get_B_field(0, 0)[1], self.B_max)
#         assert np.isclose(
#              self.coil.get_B_field(0, 0)[1] / self.coil.get_B_field(0, self.Lz/2.0)[1],
#             self.R
#         )
#         B_min = self.B_max/self.R
#         assert np.isclose(self.coil.get_B_field(0, self.Lz/2.0)[1], B_min)
#         w_ce = util.cyclotron_freq(util.constants.m_e, self.B_max)
#         assert np.isclose(w_pe / w_ce, self.w_pe_to_w_ce)
        
        print("Starting simulation with parameters:")
        print(f"    Lz = {self.Lz}m, Lx = {self.Lx}m")
        print(f"    Nz = {self.Nz}, Nx = {self.Nx}")
        print(f"    dz = {self.dz*1e6:.1f}um, dx = {self.dx*1e6:.1f}um")
        print(f"    Debye length = {self.debye_length*1e6:.1f} um")
        print(f"    Electron inertial length = {self.d_e*1e6:.1f} um")
        print(f"    Electron plasma frequency = {w_pe:.1f} Hz")
        print(f"    T_e = T_i = {self.Te:.3f} eV")
        print(f"    n0 = {self.n0:.1e} m^-3")
        print(f"    M/m = {self.m_ion/util.constants.m_e:.0f}")
        # print(f"    flux_e = {self.flux_e*util.constants.e:.1f} A/m2")
        print(f"    flux_i = {self.flux_i*util.constants.e:.1f} A/m2")
        print(f"    Ion crossing time: {ion_crossing_time}")
        print(f"    Ion crossing steps: {ion_crossing_time / self.dt:.0f}")
        print(f"    Total steps = {self.total_steps}")
mirror = MagneticMirror2D()

num_cores = 2**np.arange(8)
total_time = np.round(mirror.total_steps * 6 / num_cores / 3600, 1) # in hours
pd.DataFrame({"Cores": num_cores, "Total Hours": total_time, "Total Days": np.round(total_time/24,1)})

Starting simulation with parameters:
    Lz = 0.0001m, Lx = 3e-05m
    Nz = 32, Nx = 16
    dz = 3.1um, dx = 1.9um
    Debye length = 4.7 um
    Electron inertial length = 46.6 um
    Electron plasma frequency = 6437281239426.3 Hz
    T_e = T_i = 5150.220 eV
    n0 = 1.3e+22 m^-3
    M/m = 400
    flux_i = 4439395144.8 A/m2
    Ion crossing time: 4.699033011773388e-11
    Ion crossing steps: 3200
    Total steps = 100


,Cores,Total Hours,Total Days
0,1,0.2,0.0
1,2,0.1,0.0
2,4,0.0,0.0
3,8,0.0,0.0
4,16,0.0,0.0
5,32,0.0,0.0
6,64,0.0,0.0
7,128,0.0,0.0


In [13]:
0.1 * 60

6.0